In [32]:
import numpy as np
from ete3 import Tree
import pandas as pd

C:\Users\lenab\Anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\lenab\Anaconda3\lib\site-packages\ete3-3.1.2-py3.7.egg\ete3\evol\parser\codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


# Distance matrix

In [34]:
#an improved version of the read_subst_mtrx from project_3
#returns the matrix and a dictionary mapping the species name to indeces
def read_mtrx(filename):
    f = open(filename,'r')
    num_species = int(f.readline())

    dict_mat = {}
    mat = np.zeros((num_species,num_species))
    
    for i in range(0,num_species):
        line = f.readline()
        nums_in_line = line.split()
        dict_mat[nums_in_line[0]] = i
        for j in range(1,num_species +1):
            mat[i,j-1] = nums_in_line[j]
    f.close()
    return dict_mat, mat 

In [36]:
species_dict, dist = read_mtrx("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/example_slide4.phy")
print(species_dict)
print(dist)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
[[0.   0.23 0.16 0.2  0.17]
 [0.23 0.   0.23 0.17 0.24]
 [0.16 0.23 0.   0.2  0.11]
 [0.2  0.17 0.2  0.   0.21]
 [0.17 0.24 0.11 0.21 0.  ]]


In [43]:
list(species_dict.keys())

['A', 'B', 'C', 'D', 'E']

I propose to stor it in a database, I think it would be easier to remove columns and rows 

In [ ]:
dict_dist= {}
for i in range(len(species)) :
    dict_dist[species[i]] = dist[i]
dict_dist

In [ ]:
df=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(df)

In [ ]:
df.drop(columns="A")

In [ ]:
df.drop("A")

# Initialize a star like tree

In [76]:
ntree = "("
species = list(species_dict.keys())
for i in range(len(species)-1):
    ntree += str(species[i]) + ","
ntree += str(species[len(species)-1])+")"
print(ntree)

(A,B,C,D,E)


In [61]:
t = Tree(ntree, format=1)
print(t)

NewickError: Unexisting tree file or Malformed newick tree structure.
You may want to check other newick loading flags like 'format' or 'quoted_node_names'.

In [53]:
t = Tree("(A,B,C);", format=1)
print(t)


   /-A
  |
--|--B
  |
   \-C


In [80]:
t2 = Tree("(A,B,C,D,E);", format=1)

In [72]:
from Bio import Phylo
from io import StringIO
tree = Phylo.read(StringIO("(A, B, C, D, E)"), "newick")
print(tree)

Tree(rooted=False, weight=1.0)
    Clade()
        Clade(name='A')
        Clade(name='B')
        Clade(name='C')
        Clade(name='D')
        Clade(name='E')


In [78]:
t1 = Phylo.read(StringIO(ntree), "newick")
print(t1)

Tree(rooted=False, weight=1.0)
    Clade()
        Clade(name='A')
        Clade(name='B')
        Clade(name='C')
        Clade(name='D')
        Clade(name='E')


# NJ algorithm

## Step 1 finding the two neighboors

In [29]:
dist.shape[0]

5

In [18]:
new_N = np.full([5,5], None)
new_N

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

In [24]:
ris = []
for i in range(5):
    ri = 0
    for j in range(5):
        ri +=dist[i,j]
    ris.append(ri/3)
ris

[0.19666666666666668,
 0.21,
 0.19666666666666668,
 0.19000000000000003,
 0.24333333333333332]

In [25]:
rjs = []
for j in range(5):
    rj = 0
    for i in range(5):
        rj +=dist[i,j]
    rjs.append(rj/3)
rjs

[0.25333333333333335, 0.29, 0.23333333333333336, 0.26, 0.0]

In [28]:
minN = dist[0,0]+ris[0]+rjs[0]
mini = 0
minj = 0
for i in range(5):
    for j in range(5):
        nij = dist[i,j]+ris[i]+rjs[j]
        new_N[i,j]=nij
        if nij<minN : 
            minN = nij
            mini = i
            minj = j
print(new_N)
print(mini,minj,minN)

[[0.45000000000000007 0.7166666666666667 0.5900000000000001
  0.6566666666666667 0.19666666666666668]
 [0.6933333333333334 0.5 0.6733333333333333 0.64 0.21]
 [0.6100000000000001 0.7166666666666667 0.43000000000000005
  0.6566666666666667 0.19666666666666668]
 [0.6433333333333333 0.65 0.6233333333333334 0.45000000000000007
  0.19000000000000003]
 [0.6666666666666667 0.7733333333333332 0.5866666666666667
  0.7133333333333334 0.24333333333333332]]
3 4 0.19000000000000003


In [116]:
def step1(dist):
    S = dist.shape[0]
    new_N = np.full([S,S], None)
    ris = []
    for i in range(S):
        ri = 0
        for j in range(S):
            ri +=dist[i,j]
        ris.append(ri/(S-2))
    
    rjs = []
    for j in range(S):
        rj = 0
        for i in range(S):
            rj +=dist[i,j]
        rjs.append(rj/(S-2))
        
    minN = dist[0,0]-ris[0]-rjs[0]
    mini = 0
    minj = 0
    for i in range(5):
        for j in range(5):
            nij = dist[i,j]+ris[i]+rjs[j]
            new_N[i,j]=nij
            if nij<minN : 
                minN = nij
                mini = i
                minj = j
    return(mini,minj,minN)

In [117]:
step1(dist)

(0, 0, -0.5066666666666667)

## Step 2 : add node k

In [ ]:
species_dict, dist = read_mtrx("C:/Users/lenab/Documents/AU/Algorithms_in_bioinformatics/Brouillons_projets/example_slide4.phy")
print(species_dict)
print(dist)

In [104]:
species = list(species_dict.keys())
species

['A', 'B', 'C', 'D', 'E']

In [111]:
node_list = list(species_dict.keys())

In [115]:
node_list = list(species_dict.keys())
node_list.remove(species[mini])
node_list.remove(species[minj])
node_list.append('('+species[mini]+','+species[minj]+')')
node_list

['A', 'B', 'C', '(D,E)']

In [ ]:
def step2(mini,minj,node_list):
    nodes = node_list.copy()
    node_list.remove(nodes[mini])
    node_list.remove(nodes[minj])
    node_list.append('('+nodes[mini]+','+nodes[minj]+')')
    return node_list

## Step 3

In [119]:
df.drop(node_list[])

    